# Dataosuces declaration

## IMDB Datasource

In [1]:
from imdb import IMDb
imdb = IMDb()
[ (x.data['title'], x.data['rating']) for x in imdb.get_top250_movies() ][0:25]

[('The Shawshank Redemption', 9.2),
 ('The Godfather', 9.2),
 ('The Godfather: Part II', 9.0),
 ('The Dark Knight', 9.0),
 ('12 Angry Men', 8.9),
 ("Schindler's List", 8.9),
 ('The Lord of the Rings: The Return of the King', 8.9),
 ('Pulp Fiction', 8.9),
 ('The Good, the Bad and the Ugly', 8.8),
 ('Fight Club', 8.8),
 ('The Lord of the Rings: The Fellowship of the Ring', 8.8),
 ('Forrest Gump', 8.7),
 ('Star Wars: Episode V - The Empire Strikes Back', 8.7),
 ('Inception', 8.7),
 ('The Lord of the Rings: The Two Towers', 8.7),
 ("One Flew Over the Cuckoo's Nest", 8.7),
 ('Goodfellas', 8.7),
 ('The Matrix', 8.6),
 ('Seven Samurai', 8.6),
 ('Se7en', 8.6),
 ('City of God', 8.6),
 ('Star Wars: Episode IV - A New Hope', 8.6),
 ('The Silence of the Lambs', 8.6),
 ("It's a Wonderful Life", 8.6),
 ('Life Is Beautiful', 8.6)]

## MovieLens Datasource

In [2]:
import urllib.request
import zipfile

url = 'http://files.grouplens.org/datasets/movielens/ml-20m.zip'
download_path = './data/ml-20m.zip'
extract_path = './data/'

# urllib.request.urlretrieve(url, download_path)

# with zipfile.ZipFile(download_path, 'r') as zip:
#     zip.extractall(extract_path)

In [3]:
datasets = {
    'genome-scores': './data/ml-20m/genome-scores.csv',
    'genome-tags': './data/ml-20m/genome-tags.csv',
    'links': './data/ml-20m/links.csv',
    'movie': './data/ml-20m/movies.csv',
    'ratings': './data/ml-20m/ratings.csv',
    'tags': './data/ml-20m/tags.csv',
}

# Data Profiling

In [4]:
import pandas_profiling
import pandas as pd

def read(dataset):
    print('Reading: ', datasets[dataset])
    return pd.read_csv(datasets[dataset])

In [5]:
from IPython.display import display

for dataset_k in datasets:
    df = read(dataset_k)
    display(df.head())

Reading:  ./data/ml-20m/genome-scores.csv


,movieId,tagId,relevance
0,1,1,0.02500
1,1,2,0.02500
2,1,3,0.05775
3,1,4,0.09675
4,1,5,0.14675


Reading:  ./data/ml-20m/genome-tags.csv


,tagId,tag
0,1,007
1,2,007 (series)
2,3,18th century
3,4,1920s
4,5,1930s


Reading:  ./data/ml-20m/links.csv


...

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


Reading:  ./data/ml-20m/ratings.csv


,userId,movieId,rating,timestamp
0,1,2,3.5,1112486027
1,1,29,3.5,1112484676
2,1,32,3.5,1112484819
3,1,47,3.5,1112484727
4,1,50,3.5,1112484580


Reading:  ./data/ml-20m/tags.csv


,userId,movieId,tag,timestamp
0,18,4141,Mark Waters,1240597180
1,65,208,dark hero,1368150078
2,65,353,dark hero,1368150079
3,65,521,noir thriller,1368149983
4,65,592,dark hero,1368150078


In [17]:
genome_tags = read('genome-tags')
tags = read('tags')

genome_tags.merge(tags, on='tag').groupby(['tagId','tag']).mean()

Reading:  ./data/ml-20m/genome-tags.csv
Reading:  ./data/ml-20m/tags.csv


,,userId,movieId,timestamp
tagId,tag,,,
1,007,65271.814103,10511.923077,1.202218e+09
2,007 (series),34201.754098,25721.852459,1.275241e+09
3,18th century,63920.747253,22978.197802,1.284943e+09
4,1920s,77866.673913,51179.347826,1.303182e+09
5,1930s,70185.000000,30856.907801,1.300783e+09
6,1950s,70168.492188,26846.234375,1.298529e+09
7,1960s,74204.989474,31213.552632,1.296758e+09
8,1970s,70166.606557,32157.480874,1.308157e+09
9,1980s,65252.158076,23828.910653,1.295210e+09


# ETL

In [ ]:
def transform(*args):
    yield tuple(
        map(str.title, args)
    )

In [ ]:
def load(*args):
    print(*args)

In [7]:
import bonobo

graph = bonobo.Graph()
graph.add_chain(extract, transform, load)

GraphRange(graph=<bonobo.structs.graphs.Graph object at 0x7f5aae83fb38>, input=0, output=2)

In [8]:
bonobo.run(graph)

BonoboWidget()

ERROR:bonobo.execution.contexts.base:<NodeExecutionContext(+transform) in=1 err=1>
Traceback (most recent call last):
  File "/opt/conda/lib/python3.6/site-packages/bonobo/execution/contexts/node.py", line 102, in loop
    self.step()
  File "/opt/conda/lib/python3.6/site-packages/bonobo/execution/contexts/node.py", line 140, in step
    result = next(results)
  File "<ipython-input-6-0bb4b8f1e12a>", line 9, in transform
    map(str.title, args)
TypeError: descriptor 'title' requires a 'str' object but received a 'int'
ERROR:bonobo.execution.contexts.base:<NodeExecutionContext(+transform) in=2 err=2>
Traceback (most recent call last):
  File "/opt/conda/lib/python3.6/site-packages/bonobo/execution/contexts/node.py", line 102, in loop
    self.step()
  File "/opt/conda/lib/python3.6/site-packages/bonobo/execution/contexts/node.py", line 140, in step
    result = next(results)
  File "<ipython-input-6-0bb4b8f1e12a>", line 9, in transform
    map(str.title, args)
TypeError: descriptor 'ti